# 15 上下文管理器 和 else 块

## else块 = then

除了 if/else的组合，还有 for/else; while/else; try/else组合。

在后面这三个组合中，else更像then的概念：
- for/else ：for循环没有被break中止，完整运行完循环后，才运行else块
- while/else : 当while因为条件为假值而退出时（没有被break中止），才运行else块
- try/else : 当try没有异常抛出时才运行else块。

像极了then（但是没有then关键字！）

> 两种风格
>
> EAFP (easier to ask for forgiveness than permission)
>
> 取得原谅比获得许可容易。有很多try/except语句
>
> LBYL (look before you leap)
>
> 三思而后行. 有很多if语句。（多线程中会有风险，需要使用锁或者使用EAFP风格）

## 上下文管理器和with块

上下文管理器对象存在的目的是管理with语句，就像迭代器的存在是为了管理for语句一样。

with语句的目的是简化**try/finally模式**。

这种模式用于保证一段代码运行完毕后执行某项操作，即便那段代码由于异常、return语句或sys.exit（）调用而中止，
也会执行指定的操作。

finally子句中的代码通常用于释放重要的资源，或者还原临时变更的状态。

上下文管理器协议包含\_\_enter__和\_\_exit__两个方法。

with语句开始运行时，会在上下文管理器对象上调用\_\_enter__方法。
with语句运行结束后，会在上下文管理器对象上调用\_\_exit__方法，以此扮演finally子句的角色。

In [1]:
class LookingGlass:
    # with语句开始运行时，会在上下文管理器对象上调用\_\_enter__方法。
    def __enter__(self):
        import sys
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write # 猴子补丁！
        return 'JABBERWOCKY' # 赋值给as后面的变量

    def reverse_write(self, text):
        self.original_write(text[::-1])

    # with语句运行结束后，会在上下文管理器对象上调用\_\_exit__方法，以此扮演finally子句的角色。
    # 如果一切正常，Python调用__exit__方法时传入的参数是None, None, None；
    # 如果抛出了异常，这三个参数是异常数据
    def __exit__(self, exc_type, exc_val, exc_tb):
        import sys
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print('Please DO NOT divide by zero!')
            return True # 然后返回True，告诉解释器，异常已经处理了。
        # 如果__exit__方法返回None，或者True之外的值，with块中的任何异常都会向上冒泡.

In [3]:
with LookingGlass() as lg:
    print('Alice, Kitty and Snowdrop')
    print(lg)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [4]:
print("Back to normal!")

Back to normal!


In [5]:
lg


'JABBERWOCKY'

## 使用@contextmanager

@contextmanager装饰器能减少创建上下文管理器的样板代码量，因为不用编写一个完整的类，
定义\_\_enter__和\_\_exit__方法，而只需实现有一个yield语句的生成器，生成想让\_\_enter__方法返回的值。

在使用@contextmanager装饰的生成器中，yield语句的作用是把函数的定义体分成两部分：
- yield语句前面的所有代码在with块开始时（即解释器调用__enter__方法时）执行，
- yield语句后面的代码在with块结束时（即调用__exit__方法时）执行。

In [8]:
import contextlib
# 装饰器的一大特性是，能把被装饰的函数替换成其他函数。第二个特性是，装饰器在加载模块时立即执行。
@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    def reverse_write(text):
        original_write(text[::-1])
    sys.stdout.write = reverse_write
    print('__enter__')
    yield 'JABBERWOCKY' # 赋值给as后面的变量
    sys.stdout.write = original_write
    print('__exit__')

with looking_glass() as lg:
    print('Alice, Kitty and Snowdrop')
    print(lg)

__retne__
pordwonS dna yttiK ,ecilA
YKCOWREBBAJ
__exit__


In [7]:
lg


'JABBERWOCKY'

In [10]:
# 原来的版本如果出现异常，会导致在yield出再次抛出异常而导致后面的代码无法执行。
# 改进：

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    def reverse_write(text):
        original_write(text[::-1])
    sys.stdout.write = reverse_write
    print('__enter__')
    msg =''
    try:
        yield 'JABBERWOCKY' # 赋值给as后面的变量
    except ZeroDivisionError:
        msg = 'Please DO NOT divide by zero!'
    finally: # 无论是否出现异常，都会执行：
        sys.stdout.write = original_write
        print('__exit__')
        if msg:
            print(msg)
# 使用@contextmanager装饰器时，要把yield语句放在try/finally语句中（或者放在with语句中），
# 这是无法避免的，因为我们永远不知道上下文管理器的用户会在with块中做什么。

with looking_glass() as what:
    print(1/0)

# 使用@contextmanager装饰器时，默认的行为是相反的：
# 装饰器提供的__exit__方法假定发给生成器的所有异常都得到处理了，因此应该压制异常。
# 如果不想让@contextmanager压制异常，必须在被装饰的函数中显式重新抛出异常。

__retne__
__exit__
Please DO NOT divide by zero!
